复习私有属性

In [2]:
class Person:
    def __init__(self):
        self.__age = 10

    def get_age(self):
        return self.__age

    def set_age(self, value):
        if value < 0:
            value = 0
        self.__age = value

    def del_age(self):
        del self.__age

p = Person()
p.set_age(-10)
print(p.get_age())

0


In [3]:
p.del_age()
print(p.get_age())

AttributeError: 'Person' object has no attribute '_Person__age'

property

In [7]:
class Person:
    def __init__(self):
        self.__age = 10

    def get_age(self):
        return self.__age

    def set_age(self, value):
        if value < 0:
            value = 0
        self.__age = value

    def del_age(self):
        del self.__age

    # 外界想操作这个私有属性，是间接调用这3个方法
    age = property(get_age, set_age, del_age)    # 🔸
    
    name = "sz"  #如果加普通属性，注意help中其与age属不同分类

p = Person()
p.age = 18
print(p.age)

18


In [5]:
del p.age
print(p.age)

AttributeError: 'Person' object has no attribute '_Person__age'

In [8]:
help(Person)

Help on class Person in module __main__:

class Person(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  del_age(self)
 |  
 |  get_age(self)
 |  
 |  set_age(self, value)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  age
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  name = 'sz'



### 方法1: Property

In [9]:
class Person:
    def __init__(self):
        self.__age = 10

    @property
    def age(self):    # 注意这里改名了
        return self.__age

    @age.setter       # 注意这里装饰器的名字
    def age(self, value):
        if value < 0:
            value = 0
        self.__age = value

    @age.deleter      #
    def age(self):    #
        del self.__age

p = Person()
p.age = 18
print(p.age)
# del p.age

18


👆注意这里1个属性3个方法，那属性一多就很臃肿

### 方法2: 定义描述器类

In [16]:
class Age:
    def __get__(self, instance, owner):
        print("get")

    def __set__(self, instance, value):
        print("set")

    def __delete__(self, instance):
        print("delete")

class Person:
    age = Age()

p = Person()
p.age = 18
print(p.age) # 因为这里没定义具体值，所以get的是None
del p.age

set
get
None
delete


#### 通过实例操作描述器。通过类会出问题:

In [11]:
print(Person.age)
Person.age = 19
del Person.age

get
None


#### 不能顺利转换的场景：

#### 新式类才能起效

py3.x默认隐式继承object即为新式类，  
描述器只有在新式类中才有效，且当宿主类和描述器所对应的类，  
都为新式类时才有效。

#### 方法拦截

getattribute，如果实现了描述器的get方法，就会直接调用。  
如果没有，则按一个实例属性的访问顺序查找:

1. 实例对象自身dict 
2. 对应类对象dict  
3. （如果有）父类dict   
4. 还没找到，有getattr方法，调用这个方法

In [15]:
class Age(object):  # 这里保险起见，两者都继承object
    def __get__(self, instance, owner):
        print("get")

    def __set__(self, instance, value):
        print("set")

    def __delete__(self, instance):
        print("delete")

class Person(object):  # 即使py2.x也是新式类
    age = Age()
    def __getattribute__(self, item):  #🔸方法拦截
        print("xxxxx")

p = Person()
p.age = 18 # 未调用get方法，而是走getattribute
print(p.age)
del p.age

set
xxxxx
None
delete


资料描述器 get set  
非资料描述器 仅仅实现了 get 方法，那他就是一个非资料描述器  
资料描述器 > 实例属性 > 非资料描述器

In [21]:
class Age:   # 有get set，是资料描述器
    def __get__(self, instance, owner):
        print("get")

    def __set__(self, instance, value):
        print("set")

    def __delete__(self, instance):
        print("delete")

class Person(object):  
    age = Age()
    def __init__(self): # 注意这里
        self.age = 10

p = Person()   # init=>set
p.age = 10     # set
print(p.age)   # get None
print(p.__dict__) # {}

set
set
get
None
{}


In [22]:
class Age:   # 只有get 是非资料描述器
    def __get__(self, instance, owner):
        print("get")

class Person(object):  
    age = Age()
    def __init__(self): # 注意这里
        self.age = 10

p = Person()   # init=>实例属性
p.age = 10     # 修改age
print(p.age)   # 10
print(p.__dict__) 

10
{'age': 10}


#### 存储问题

In [24]:
class Age:   
    def __get__(self, instance, owner):
        print("get")

    def __set__(self, instance, value):
        print("set", self, instance, value) # 注意这里

    def __delete__(self, instance):
        print("delete")

class Person(object):  
    age = Age()

p1 = Person()   
p1.age = 10

p2 = Person()   
p2.age = 12

set <__main__.Age object at 0x0000024154C5CBE0> <__main__.Person object at 0x00000241549FA970> 10
set <__main__.Age object at 0x0000024154C5CBE0> <__main__.Person object at 0x00000241549FAEE0> 12


#### Age对象是共享的

In [29]:
class Age:   
    def __get__(self, instance, owner):
        print("get")
        return self.v      # 注意这里

    def __set__(self, instance, value):
        print("set", self, instance, value)
        self.v = value      # 注意这里

    def __delete__(self, instance):
        print("delete")

class Person(object):  
    age = Age()

p1 = Person()   
p1.age = 10
print(p1.age)
p2 = Person()   
p2.age = 12
print(p2.age)
print(p1.age) # 修改其中任何一个，都改了
# 我们的本意是不同实例对象，不同属性数据
# 所以不能绑定在self身上，应该绑在instantce身上

set <__main__.Age object at 0x0000024154C6E820> <__main__.Person object at 0x0000024154C6E190> 10
get
10
set <__main__.Age object at 0x0000024154C6E820> <__main__.Person object at 0x00000241549FA400> 12
get
12
get
12
